In [0]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re
import io
import requests
import csv

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
from ast import literal_eval
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


## Importing AirBnb data

### Reviews

In [0]:
#Import review data
url_r = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/reviews/reviews_15_7_19.csv'
reviews = pd.read_csv(url_r)

### Listings

In [0]:
#Import listings data from each scraping iteration (from oldest to newest)
url_l1 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_4_15.csv'
url_l2 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_5_18.csv'
url_l3 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_7_18.csv'
url_l4 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_9_14.csv'
url_l5 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_10_11.csv'
url_l6 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_11_15.csv'
url_l7 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_18_12_13.csv'
url_l8 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_1_17.csv'
url_l9 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_2_9.csv'
url_l10 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_3_12.csv'
url_l11 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_4_15.csv'
url_l12 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_5_19.csv'
url_l13 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_6_14.csv'
url_l14 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/listings/listings_19_7_15.csv'


listings_1 = pd.read_csv(url_l1)
listings_2 = pd.read_csv(url_l2)
listings_3 = pd.read_csv(url_l3)
listings_4 = pd.read_csv(url_l4)
listings_5 = pd.read_csv(url_l5)
listings_6 = pd.read_csv(url_l6)
listings_7 = pd.read_csv(url_l7)
listings_8 = pd.read_csv(url_l8)
listings_9 = pd.read_csv(url_l9)
listings_10 = pd.read_csv(url_l10)
listings_11 = pd.read_csv(url_l11)
listings_12 = pd.read_csv(url_l12)
listings_13 = pd.read_csv(url_l13)
listings_14 = pd.read_csv(url_l14)

In [0]:
#The most recent listing data is the one we want, but some past listings may no longer show up
#We'll append to the most recent listings data from past scrapings, but we'll only keep the most recent information for each id 
listings=listings_14
listings=listings.append(listings_13)
listings=listings.append(listings_12)
listings=listings.append(listings_11)
listings=listings.append(listings_10)
listings=listings.append(listings_9)
listings=listings.append(listings_8)
listings=listings.append(listings_7)
listings=listings.append(listings_6)
listings=listings.append(listings_5)
listings=listings.append(listings_4)
listings=listings.append(listings_3)
listings=listings.append(listings_2)
listings=listings.append(listings_1)

listings=listings.drop_duplicates(subset="id", keep='first')
listings=listings.drop(columns=['name','host_name','price','minimum_nights','neighbourhood_group'])
listings=listings.dropna(subset=['last_review'], axis=0)
listings['lr_m']=listings.last_review.apply(lambda x: int(x[5:7]))
listings['lr_d']=listings.last_review.apply(lambda x: int(x[8:10]))
listings['lr_y']=listings.last_review.apply(lambda x: int(x[0:4]))
listings.last_review = pd.to_datetime(listings.last_review)
listings['lat']=listings.latitude.round(2)
listings['lon']=listings.longitude.round(2)
listings['location'] = list(zip(listings.latitude, listings.longitude))
listings['loc'] = list(zip(listings.lat, listings.lon))
listings = listings.assign(loc_id=(listings['loc'].astype('category').cat.codes))
listings.room_type = listings.room_type.apply(lambda x: 1 if x=="Entire home/apt" else 2 if x=="Private room" else 3)

listings_sel=listings[listings.number_of_reviews > 9]
listings_sel=listings_sel[listings_sel.lr_y > 2017]
listings_sel=listings_sel[listings_sel.availability_365>9]
listings_sel=listings_sel.drop(listings_sel[(listings_sel.lr_y==2018) & (listings_sel.lr_m<4)].index)
listings_sel=listings_sel.drop(listings_sel[(listings_sel.lr_y==2018) & (listings_sel.lr_m==4) & (listings_sel.lr_d<15)].index)

In [6]:
listings_locations = listings_sel[['loc','loc_id']]
listings_locations = listings_locations.drop_duplicates('loc_id')
listings_locations = listings_locations.rename(index=str, columns={"loc": "list_loc","loc_id": "list_loc_id"})
listings_locations=listings_locations.set_index('list_loc_id')
listings_locations.sort_index()
list_locs=list(listings_locations['list_loc'])
len(list_locs)

370

### Calendar

In [0]:
#Import calendar data from each scraping iteration (from oldest to newest)
url_c1 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_4_15.zip'
url_c2 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_5_18.zip'
url_c3 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_7_18.zip'
url_c4 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_9_14.zip'
url_c5 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_10_11.zip'
url_c6 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_11_15.zip'
url_c7 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_18_12_13.zip'
url_c8 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_1_17.zip'
url_c9 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_2_9.zip'
url_c10 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_3_12.zip'
url_c11 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_4_15.zip'
url_c12 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_5_19.zip'
url_c13 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_6_14.zip'
url_c14 = 'https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/calendar/calendar_19_7_15.zip'


calendar_1 = pd.read_csv(url_c1)
calendar_2 = pd.read_csv(url_c2)
calendar_3 = pd.read_csv(url_c3)
calendar_4 = pd.read_csv(url_c4)
calendar_5 = pd.read_csv(url_c5)
calendar_6 = pd.read_csv(url_c6)
calendar_7 = pd.read_csv(url_c7)
calendar_8 = pd.read_csv(url_c8)
calendar_9 = pd.read_csv(url_c9)
calendar_10 = pd.read_csv(url_c10)
calendar_11 = pd.read_csv(url_c11)
calendar_12 = pd.read_csv(url_c12)
calendar_13 = pd.read_csv(url_c13)
calendar_14 = pd.read_csv(url_c14)

In [0]:
#For each calendar scraping, add scraping date
calendar_1['scr_date']='2018-04-15'
calendar_2['scr_date']='2018-05-18'
calendar_3['scr_date']='2018-07-18'
calendar_4['scr_date']='2018-09-14'
calendar_5['scr_date']='2018-10-11'
calendar_6['scr_date']='2018-11-15'
calendar_7['scr_date']='2018-12-13'
calendar_8['scr_date']='2019-01-17'
calendar_9['scr_date']='2019-02-09'
calendar_10['scr_date']='2019-03-12'
calendar_11['scr_date']='2019-04-15'
calendar_12['scr_date']='2019-05-19'
calendar_13['scr_date']='2019-06-14'
calendar_14['scr_date']='2019-07-15'

In [9]:
calendar=calendar_14
calendar=calendar.append(calendar_13)
calendar=calendar.append(calendar_12)
calendar=calendar.append(calendar_11)
calendar=calendar.append(calendar_10)
calendar=calendar.append(calendar_9)
calendar=calendar.append(calendar_8)
calendar=calendar.append(calendar_7)
calendar=calendar.append(calendar_6)
calendar=calendar.append(calendar_5)
calendar=calendar.append(calendar_4)
calendar=calendar.append(calendar_3)
calendar=calendar.append(calendar_2)
calendar=calendar.append(calendar_1)

calendar=calendar[['listing_id','date','price','scr_date']]
calendar=calendar.drop_duplicates(subset=['listing_id','date','price'])
calendar=calendar.dropna(axis=0,subset=['price'])
calendar['month']=calendar.date.apply(lambda x: int(x[5:7]))
calendar['day']=calendar.date.apply(lambda x: int(x[8:10]))
calendar['year']=calendar.date.apply(lambda x: int(x[0:4]))
calendar.date = pd.to_datetime(calendar.date)
calendar.scr_date = pd.to_datetime(calendar.scr_date)
calendar.price = calendar.price.apply(lambda x: float(re.sub("[^\d\.]", "", (x[1:-3]))))
calendar.price = pd.to_numeric(calendar.price)

cal_change = calendar[calendar.duplicated(['listing_id','date'],keep=False)]
cal_change.sort_values(by=['listing_id','date'])

calendar=calendar.drop_duplicates(subset=['listing_id','date'])

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [10]:
cal_change.sort_values(by=['listing_id','date'])

,listing_id,date,price,scr_date,month,day,year
52170,2384,2018-06-11,65.0,2018-05-18,6,11,2018
70950,2384,2018-06-11,80.0,2018-04-15,6,11,2018
52093,2384,2018-07-01,65.0,2018-05-18,7,1,2018
70930,2384,2018-07-01,60.0,2018-04-15,7,1,2018
52092,2384,2018-07-02,65.0,2018-05-18,7,2,2018
70929,2384,2018-07-02,60.0,2018-04-15,7,2,2018
2546,2384,2018-07-26,75.0,2018-07-18,7,26,2018
52068,2384,2018-07-26,65.0,2018-05-18,7,26,2018
70905,2384,2018-07-26,60.0,2018-04-15,7,26,2018
52067,2384,2018-07-27,65.0,2018-05-18,7,27,2018


## Importing crimes data

In [0]:
url_cr = "https://raw.githubusercontent.com/araujoghm/DataScienceEMAp_AraujoNovais/master/dados/crimes/crimes.csv"
crimes = pd.read_csv(url_cr)
crimes = crimes[['CASE#','DATE  OF OCCURRENCE','LATITUDE','LONGITUDE','ARREST',' PRIMARY DESCRIPTION']]
crimes = crimes.rename(index=str, columns={"CASE#": "case_id","DATE  OF OCCURRENCE": "date","LONGITUDE": "longitude","LATITUDE": "latitude"," PRIMARY DESCRIPTION": "desc", "ARREST": "arrest"})
crimes['lat']=crimes.latitude.round(2)
crimes['lon']=crimes.longitude.round(2)
crimes['location'] = list(zip(crimes.latitude, crimes.longitude))
crimes['loc'] = list(zip(crimes.lat, crimes.lon))
crimes = crimes.assign(loc_id=(crimes['loc'].astype('category').cat.codes))
crimes.arrest = crimes.arrest.apply(lambda x: 0 if x=="N" else 1)
crimes.date = crimes.date.apply(lambda x: x[0:10])
crimes.date = pd.to_datetime(crimes.date)
crimes=crimes.dropna(axis=0)

In [12]:
crimes_locations = crimes[['location','loc_id']]
crimes_locations = crimes_locations.drop_duplicates('loc_id')
crimes_locations = crimes_locations.rename(index=str, columns={"location": "crim_loc","loc_id": "crim_loc_id"})
crimes_locations=crimes_locations.set_index('crim_loc_id')
crimes_locations.sort_index()
crim_locs=list(crimes_locations['crim_loc'])
len(crim_locs)

710

## Calculating distances between Airbnb listings and criminal occurrences

We're interested in knowing the criminal activity surrounding each Airbnb listing. To do that, we'll calculate the distance between each unique location on our listings database and each unique location on our criminal occurrence database.

In [0]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [0]:
def distance(a,b):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees). Output in KM
    """
    lat1 = a[0]
    lat2 = b[0]
    lon1 = a[1]
    lon2 = b[1]
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    km = 6371 * c
    return km

In [0]:
distances=[None]*len(list_locs)
for i in range(len(list_locs)):
  a=[None]*len(crim_locs)
  x=list_locs[i]
  for j in range(len(crim_locs)):
    a[j]=round(distance(x,crim_locs[j]),2)
  distances[i]=a
  
dist_df=pd.DataFrame(distances)

In [19]:
dist_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709
0,12.02,10.03,6.41,12.15,2.67,13.17,9.24,21.08,18.26,5.10,3.38,4.82,8.74,14.38,14.77,17.28,2.52,18.13,5.98,10.48,16.65,14.30,20.13,13.26,7.33,3.88,9.42,11.30,17.70,33.31,14.66,22.67,26.32,17.16,10.14,12.48,18.28,14.26,9.20,17.12,...,28.70,10.29,14.37,11.68,13.71,16.30,32.77,25.67,14.99,12.71,15.03,13.00,12.12,12.99,12.43,22.41,32.50,33.25,35.08,11.96,35.55,31.60,16.31,16.01,12.09,28.01,33.22,30.99,2.97,29.48,13.65,14.58,13.12,35.04,32.17,30.35,30.69,674.44,12.07,16.19
1,7.73,8.55,11.77,17.93,9.66,8.29,14.84,13.38,11.48,14.11,13.72,12.25,3.65,3.49,22.79,5.99,8.98,6.84,12.56,16.46,5.36,3.01,10.20,2.11,10.78,15.09,6.54,9.04,6.45,22.03,6.58,13.52,18.87,8.01,1.53,7.63,6.99,4.18,16.77,10.69,...,17.52,19.07,24.35,20.97,13.21,18.59,21.47,16.42,10.21,12.96,23.38,22.22,21.89,22.13,22.55,11.12,21.22,21.96,23.86,4.01,24.27,21.04,23.88,17.84,22.74,18.19,22.06,19.70,9.47,19.46,22.08,23.53,3.49,23.87,20.87,19.29,19.47,675.20,21.34,24.25
2,4.34,14.28,16.72,23.20,13.61,4.05,20.00,7.66,5.69,18.33,17.39,16.74,9.11,3.77,27.65,7.07,12.60,7.34,17.31,21.63,6.64,5.35,5.88,6.31,16.01,18.17,5.56,14.84,6.26,20.46,0.82,8.44,13.19,2.84,7.14,13.39,7.73,2.27,21.57,4.99,...,15.46,23.60,28.46,25.35,19.01,24.33,20.29,11.45,15.83,18.73,28.16,26.66,26.07,26.60,26.56,10.49,20.35,20.45,21.80,9.68,22.62,17.40,28.89,23.60,26.40,13.69,19.76,18.37,11.66,15.15,26.80,28.10,8.66,21.57,19.69,16.70,17.49,680.80,25.74,29.13
3,3.02,17.27,18.74,25.36,15.02,1.88,22.11,6.14,3.23,19.66,18.35,18.36,11.99,7.75,29.31,10.97,13.88,11.14,19.14,23.71,10.58,9.31,7.53,10.15,18.32,18.65,5.64,18.00,10.02,22.79,3.23,8.46,11.29,4.75,10.32,16.84,11.55,6.25,23.31,2.09,...,17.72,25.02,29.41,26.60,22.15,27.20,22.84,11.26,19.44,21.74,29.72,27.93,27.14,27.90,27.46,13.76,23.06,22.82,23.80,13.24,24.85,18.36,30.69,26.53,27.01,13.96,21.67,20.91,12.10,15.57,28.34,29.46,12.42,23.43,22.27,18.54,19.70,684.43,27.00,30.79
4,10.95,10.29,7.20,13.16,3.08,12.10,10.17,20.04,17.20,6.19,4.49,5.75,8.41,13.64,15.88,16.67,2.46,17.50,6.88,11.47,16.03,13.65,19.23,12.69,7.95,4.78,8.37,11.56,17.01,32.63,13.69,21.68,25.25,16.22,9.61,12.54,17.67,13.44,10.27,16.06,...,27.96,11.41,15.44,12.78,14.24,17.12,32.13,24.68,15.16,13.28,16.14,14.11,13.21,14.10,13.49,21.76,31.89,32.57,34.35,11.69,34.87,30.71,17.41,16.80,13.10,27.05,32.46,30.33,1.88,28.54,14.76,15.69,12.74,34.29,31.52,29.56,29.96,675.38,13.18,17.31
5,2.00,16.22,17.62,24.25,13.92,0.94,21.00,7.14,4.29,18.56,17.28,17.25,10.95,7.16,28.21,10.56,12.78,10.82,18.02,22.60,10.13,8.66,7.92,9.37,17.21,17.61,4.56,16.98,9.72,23.13,2.87,9.24,12.37,4.93,9.33,15.88,11.22,5.69,22.20,3.16,...,18.06,23.92,28.34,25.51,21.12,26.12,23.11,12.12,18.52,20.69,28.61,26.84,26.06,26.81,26.40,13.73,23.27,23.14,24.23,12.33,25.22,19.04,29.58,25.46,25.98,14.76,22.13,21.17,11.05,16.34,27.23,28.37,11.60,23.90,22.52,19.00,20.06,683.49,25.91,29.68
6,4.76,17.88,19.86,26.45,16.37,3.70,23.21,4.48,1.99,21.06,19.87,19.65,12.64,7.31,30.62,10.06,15.26,10.09,20.35,24.83,9.77,8.93,5.66,10.00,19.31,20.31,7.20,18.49,8.94,20.91,2.92,6.49,9.90,3.21,10.77,17.11,10.51,5.85,24.57,1.34,...,15.83,26.41,30.96,28.06,22.66,27.90,21.00,9.35,19.55,22.35,31.07,29.38,28.66,29.34,29.03,12.24,21.26,20.93,21.85,13.40,22.93,16.37,31.95,27.19,28.66,12.00,19.72,19.07,13.72,13.60,29.69,30.88,12.37,21.47,20.44,16.59,17.80,684.51,28.45,32.10
7,9.10,22.09,24.36,30.91,20.89,7.96,27.69,0.40,2.87,25.58,24.37,24.17,16.93,10.80,35.14,12.57,19.79,12.28,24.86,29.31,12.48,12.36,5.82,13.66,23.75,24.73,11.66,22.61,11.21,19.44,7.13,3.79,5.40,5.79,14.96,21.03,12.69,9.53,29.09,4.02,...,14.56,30.93,35.45,32.57,26.77,32.15,19.85,5.75,23.27,26.53,35.59,33.90,33.15,33.86,33.51,12.98,20.35,19.51,19.89,17.28,21.23,13.44,36.46,31.41,33.09,8.63,17.74,17.99,18.17,10.25,34.22,35.41,16.04,19.33